In [1]:
import numpy as np

from numba import njit

In [2]:
def show(G):
    
    tmp = 'X | '
    for v in V:
        tmp += ' ' + v + ' '
    
    print(tmp)
    print('')
    
    for row in range(len(V)):
        
        tmp = V[row] + ' | '
        for col in range(Vsize):
            w = int(G[row][col])
            if w < 10 : tmp +=  ' ' + str(w) + ' '
            if 10 <= w < 100 : tmp +=  ' ' + str(w)
            
            
        print(tmp)

In [3]:
def show_path(path, V):
    tmp = '[show_path] Path is: '
    for v in path:
        tmp += str(V[v]) + ','
    print(tmp, ' .')

---

## Graph init

<img src="TestGraph.png">

In [4]:
Vsize = 10
Esize = 12

V = ['S', 'a', 'b', 'c', 'd', 'x', 'y', 'z', 'g', 'E']

In [5]:
G = np.zeros((Vsize, Vsize))

In [6]:
G[0][1] = 20
G[0][2] = 30

G[1][3] = 15
G[1][4] = 10

G[2][4] = 15

G[3][5] = 15

G[4][6] = 30
G[4][7] = 10

G[5][8] = 20

G[6][9] = 25

G[7][9] = 20

G[8][9] = 5

In [7]:
show(G)

X |  S  a  b  c  d  x  y  z  g  E 

S |  0  20 30 0  0  0  0  0  0  0 
a |  0  0  0  15 10 0  0  0  0  0 
b |  0  0  0  0  15 0  0  0  0  0 
c |  0  0  0  0  0  15 0  0  0  0 
d |  0  0  0  0  0  0  30 10 0  0 
x |  0  0  0  0  0  0  0  0  20 0 
y |  0  0  0  0  0  0  0  0  0  25
z |  0  0  0  0  0  0  0  0  0  20
g |  0  0  0  0  0  0  0  0  0  5 
E |  0  0  0  0  0  0  0  0  0  0 


---

## MaxFlow

In [8]:
#@njit
def iteration(G, F, V, _inf=99999999):
    
    Prev = np.full(Vsize, -1) # Prev node
    Q = [0] #Queue
    
    
    # BFS
    while Q != []:

        v_curr = Q[0]
        for v_ind, capacity in enumerate( G[v_curr] ):
            if F[v_curr][v_ind] < capacity and Prev[v_ind] == -1 and v_ind != v_curr:
                Q.append(v_ind)
                Prev[v_ind] = v_curr
        Q.pop(0)
    
    
    # path reconstruction
    v_curr = Vsize - 1
    path = [v_curr]
    while True:
        v_prev = Prev[v_curr]
        v_curr = v_prev
        if v_curr == -1 : break
        path.append(v_curr)

    path = path[::-1]
    
    
    # path bottleneck
    bottleneck = _inf # float('inf') doesn't work with numba
    for i in range(len(path) - 1):
        v_curr = path[i]
        v_next = path[i+1]

        if G[v_curr][v_next] < bottleneck: 
            bottleneck = G[v_curr][v_next]
    if bottleneck == _inf : return F, True
    
    
    # path flow update
    for i in range(len(path) - 1):
        v_curr = path[i]
        v_next = path[i+1]

        F[v_curr][v_next] += bottleneck
        
    
    print('')
    print('----- New iteration -----')
    #show_path(path, V)
    print('Bottleneck is: ', bottleneck)
    
    return F, False

In [9]:
# Init
F_ = np.zeros((Vsize, Vsize)) # Current flow

In [10]:
while True:
    F_, end = iteration(G, F_, V)
    if end : break


----- New iteration -----
Bottleneck is:  10.0

----- New iteration -----
Bottleneck is:  15.0

----- New iteration -----
Bottleneck is:  5.0


In [11]:
print('MaxFlow is: ', sum(F_[:, -1]))

MaxFlow is:  30.0


---

## MinCut

In [12]:
# BFS
Prev = np.full(Vsize, -1) # Prev node
Q = [0] # Queue
cursor = 0 # instead of popping elements of of the Q

while True:

    v_curr = Q[cursor]
    for v_ind, capacity in enumerate( G[v_curr] ):
        if 0 < F_[v_curr][v_ind] < capacity and Prev[v_ind] == -1 and v_ind != v_curr:
            Q.append(v_ind)
            Prev[v_ind] = v_curr
    
    cursor += 1
    if cursor >= len(Q) : break
    
Q

[0, 1, 2, 3, 5, 8]

In [13]:
# S a b c d x y z g E
# 0 1 2 3 4 5 6 7 8 9